# Installing desired version of fastai - pytorch

In [0]:
!pip install fastai==1.0.54 > /dev/null

# Setting up Environment to get reproducible results

[** Didn't set num_workers as 1]

Ref : https://docs.fast.ai/dev/test.html#getting-reproducible-results

In [0]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

# Loading required packages

(Some packages can be removed, as I have carried out other operations that are not part of this release)

In [0]:
import torch.nn as nn

import torch
import torch.nn as nn
import torch.nn.functional as F
import shutil as sh
import os

from fastai.layers import *
from fastai.torch_core import *

from fastai.vision import *
from fastai.callbacks.csv_logger import CSVLogger
from fastai.callbacks.hooks import *

torch.backends.cudnn.benchmark = True

from fastai.torch_core import *
from fastai.callback import *
from fastai.callbacks import *
from fastai.basic_train import *
from fastai.basic_data import *

# Downloading CIFAR100 Dataset

In [0]:
path = untar_data(URLs.CIFAR_100)

# Parameters and Dataloader

In [0]:
bs = 128
epochs = 50
metrics = [accuracy, error_rate, top_k_accuracy]

loss_func = nn.CrossEntropyLoss()

base_lr = 0.1
momentum= 0.9
w_decay = 0.0001

opt_func = partial (optim.SGD, momentum=0.9)

result_path = '/content/sample_data/activation/results/'
final_path = "/content/drive/'My Drive'/activation/"

In [0]:
ds_tfms = ([ *rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', bs=bs ,ds_tfms = ds_tfms , val_bs =100).normalize(cifar_stats)

# Modified NIN network

Official Implementation

https://gist.github.com/mavenlin/e56253735ef32c3c296d#file-solver-prototxt

Our implementation is based on 

https://github.com/jiecaoyu/pytorch-nin-cifar10

In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.num_classes = 100
    self.classifier = nn.Sequential(
                      nn.Conv2d(3, 192, kernel_size=5, stride=1, padding=2),
                      nn.ReLU(inplace=True),
                      nn.Conv2d(192, 160, kernel_size=1, stride=1, padding=0),
                      nn.ReLU(inplace=True),
                      nn.Conv2d(160,  96, kernel_size=1, stride=1, padding=0),
                      nn.ReLU(inplace=True),
                      nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
                      nn.Dropout(0.5),

                      nn.Conv2d(96, 192, kernel_size=5, stride=1, padding=2),
                      nn.ReLU(inplace=True),
                      nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
                      nn.ReLU(inplace=True),
                      nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
                      nn.ReLU(inplace=True),
                      nn.AvgPool2d(kernel_size=3, stride=2, padding=1),
                      nn.Dropout(0.5),

                      nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1),
                      nn.ReLU(inplace=True),
                      nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
                      nn.ReLU(inplace=True),
                      nn.Conv2d(192,  self.num_classes , kernel_size=1, stride=1, padding=0),
                      nn.ReLU(inplace=True),
                      nn.AvgPool2d(kernel_size=8, stride=1, padding=0),
                      )

  def forward(self, x):
    x = self.classifier(x)
    x = x.view(x.size(0), self.num_classes)
    return x

# Initializing weights and bias

In [0]:
model = Net()
model.cuda()

pretrained=False
if pretrained:
    params = pickle.load(open('data/params', 'r'))
    index = -1
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            index = index + 1
            weight = torch.from_numpy(params[index])
            m.weight.data.copy_(weight)
            index = index + 1
            bias = torch.from_numpy(params[index])
            m.bias.data.copy_(bias)
else:
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            m.weight.data.normal_(0, 0.05)
            m.bias.data.normal_(0, 0.0)


# Splitting Model into sections for discriminative learning

In [0]:
learn = Learner(data, model, loss_func= loss_func, opt_func = opt_func ,metrics=metrics).mixup()

In [10]:
learn.split([[ learn.model.classifier[0:20]], [learn.model.classifier[20] ]])

Learner(data=ImageDataBunch;

Train: LabelList (50000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
otter,otter,otter,otter,otter
Path: /root/.fastai/data/cifar100;

Valid: LabelList (10000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
otter,otter,otter,otter,otter
Path: /root/.fastai/data/cifar100;

Test: None, model=Net(
  (classifier): Sequential(
    (0): Conv2d(3, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU(inplace)
    (2): Conv2d(192, 160, kernel_size=(1, 1), stride=(1, 1))
    (3): ReLU(inplace)
    (4): Conv2d(160, 96, kernel_size=(1, 1), stride=(1, 1))
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.5)
    (8): Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (9): ReLU(inplace)
    (10): Conv2

# Decalring Step Dropout as Callback



In [0]:
class Drop_Activations(LearnerCallback):
  train_flag = False
  
  def on_train_begin(self, **kwargs):
    "Initialize stats."
    super().on_train_begin(**kwargs)
    self.train_flag = True
    
  def on_epoch_begin(self, **kwargs):
    if (self.train_flag):
      epoch_num = kwargs['epoch']
      epoch_tot = kwargs['n_epochs']

      child = children(self.model)[0]
      for item in child:
        if isinstance(item, nn.Dropout):
          item.p = (epoch_num  / epoch_tot) * 0.5

  def on_train_end(self, **kwargs):
    "Initialize stats."
    super().on_train_begin(**kwargs)
    self.train_flag = False

# Using the step dropout in this notebook

In [0]:
learn.callbacks += [ Drop_Activations(learn) ]

# Running the Network

In [13]:
learn.fit( 40 ,  lr= (1 * base_lr , 0.1 * base_lr) , wd = (2 * w_decay , 1 *w_decay))

epoch,train_loss,valid_loss,accuracy,error_rate,top_k_accuracy,time
0,4.477996,4.432767,0.027800,0.972200,0.142900,00:34
1,4.297046,4.173907,0.070900,0.929100,0.243300,00:33
2,4.152416,3.915993,0.101400,0.898600,0.313200,00:33
3,3.982190,3.759217,0.120300,0.879700,0.351200,00:32
4,3.833276,3.510408,0.158700,0.841300,0.420000,00:33
5,3.724839,3.300355,0.203500,0.796500,0.485500,00:32
6,3.609060,3.275402,0.202400,0.797600,0.494600,00:32
7,3.468804,2.966768,0.268600,0.731400,0.576300,00:32
8,3.365316,2.747127,0.307100,0.692900,0.627200,00:32
9,3.241804,2.639326,0.328100,0.671900,0.639300,00:34


Buffered data was truncated after reaching the output size limit.

# Results

In [14]:
learn.validate()

[1.5717146, tensor(0.5796), tensor(0.4204), tensor(0.8477)]

# Best Results :

[Obtained during trial run. I hope the variation is minimal across multiple runs]

> train_loss --------------------------------- 2.445148

> valid_loss --------------------------------- 1.571715

> accuracy ---------------------------------- 0.5796

> error_rate --------------------------------- 0.4204

> top_k_accuracy -----------------------  0.8477

